In [26]:
import pandas as pd
import numpy as np
import tqdm
from os import listdir
from os.path import isfile, join
import re

# process anomoly files

In [2]:
# merge anomoly files 
path2anomoly = './data_raw/brk/20240628_2022&2023年各分點對個股異常點漲幅超過10%前五名'
files = [f for f in listdir(path2anomoly) if isfile(join(path2anomoly, f))]

brk_anomaly = pd.DataFrame()

for f in files:
    df = pd.read_csv(path2anomoly+'/'+f, sep='\t')
    df['brk'] = re.search(r"[\w]{4}", f).group(0)
    brk_anomaly = pd.concat([brk_anomaly, df], axis=0, ignore_index=True)

In [3]:
# keep 2022 data
brk_anomaly['日期'] = pd.to_datetime(brk_anomaly['日期'])
brk_anomaly_2022 = brk_anomaly[((brk_anomaly['日期']>='2022-01-01') & (brk_anomaly['日期']<='2023-01-01'))]

In [4]:
brk_anomaly_2022

,股票代號,日期,買賣超,股票漲跌幅,股票結算日,brk
0,6589,2022-11-23,-7594500,-0.113553,2022-12-21,1020
1,6589,2022-12-02,-9998014,-0.185430,2022-12-30,1020
2,6589,2022-12-07,-20176656,-0.241722,2023-01-05,1020
3,6589,2022-12-09,-15973000,-0.237013,2023-01-09,1020
7,2009,2022-03-04,-2374350,-0.126471,2022-04-01,1020
...,...,...,...,...,...,...
4464,1319,2022-05-30,8037600,0.211779,2022-06-28,700N
4465,1319,2022-06-09,8350927,0.217391,2022-07-07,700N
4466,1524,2022-05-12,8045800,0.623574,2022-06-10,700N
4467,1524,2022-09-27,-3305700,-0.194231,2022-10-26,700N


In [5]:
df_unique_anomaly_2022 = brk_anomaly_2022.drop_duplicates(subset=['股票代號', 'brk'])

# merge all data 

In [6]:
import dask.dataframe as dd
import dask.array as da

In [7]:
path2complete = './data_raw/brk/complete_file'
files = [f for f in listdir(path2complete) if isfile(join(path2complete, f))]

In [8]:
df_complete = pd.DataFrame()
ddf_complete = dd.from_pandas(df_complete, npartitions=9)

for f in files:
    df = dd.read_csv(path2complete+'/'+f)
    ddf_complete = dd.concat([ddf_complete, df])

# Raw analysis

## 1. brk magnification

**結論** : 
透過這個資料夾
20240628_2022&2023年各分點對個股異常點漲幅超過10%前五名
所抓出的分點

可以過濾掉 ""全部"" 券商資料中的 "" 70%的分點 ""

In [9]:
uq_brk_anomaly = np.unique(brk_anomaly_2022['brk'])

In [10]:
ddf_complete_len = ddf_complete.shape[0].compute()

In [11]:
brk_mask = ddf_complete['分點'].isin(uq_brk_anomaly)
ddf_complete_masked = ddf_complete[brk_mask]

In [12]:
ddf_complete_masked_len= ddf_complete_masked.shape[0].compute()

In [13]:
ddf_complete_masked_len/ddf_complete_len

0.2984408172101807

## 2. brk inner shrinkage

In [14]:
df_unique_anomaly_2022 

,股票代號,日期,買賣超,股票漲跌幅,股票結算日,brk
0,6589,2022-11-23,-7594500,-0.113553,2022-12-21,1020
7,2009,2022-03-04,-2374350,-0.126471,2022-04-01,1020
10,2405,2022-08-31,-7776250,-0.220126,2022-09-29,1020
16,2498,2022-01-03,-10223900,-0.210465,2022-02-09,1021
20,4931,2022-05-23,-8920650,-0.134054,2022-06-21,1021
...,...,...,...,...,...,...
4458,2603,2022-03-02,-29314000,-0.108911,2022-03-30,700N
4461,2615,2022-02-21,-19395280,-0.166667,2022-03-22,700N
4464,1319,2022-05-30,8037600,0.211779,2022-06-28,700N
4466,1524,2022-05-12,8045800,0.623574,2022-06-10,700N


In [19]:
enumerate_pair = {}

for brk in df_unique_anomaly_2022['brk']:
    ticker_li = list(np.unique(df_unique_anomaly_2022[df_unique_anomaly_2022['brk'] == brk]['股票代號']))
    enumerate_pair[brk] = ticker_li

In [28]:
for brk in tqdm.tqdm(enumerate_pair.keys()):

    df_brk = ddf_complete_masked[ddf_complete_masked['分點']==brk].compute()

    for tick in enumerate_pair[brk]:

        df_brk[df_brk['股號']==tick].shape[0]

100%|██████████| 293/293 [00:00<00:00, 293581.24it/s]


In [15]:
ouo = ddf_complete_masked[(ddf_complete_masked['分點']=='1020')&(ddf_complete_masked['股號']==6589)].compute()

In [34]:
ouo.shape[0]

207

In [212]:
# ddf_complete_masked[ddf_complete_masked['分點']=="1020"].shape[0].compute()

144477

In [217]:
_1020_6589 = ddf_complete_masked[(ddf_complete_masked['分點']=="1020")&(ddf_complete_masked['股號']==6589)].compute()

In [ ]:
# https://holoviews.org/user_guide/Network_Graphs.html

In [216]:
test

,日期,股號,分點,買賣超金額,是否為總公司
731673,2022-01-17,6589.0,1020,-120880.0,Y
975748,2022-01-13,6589.0,1020,16050.0,Y
752941,2022-01-03,6589.0,1020,21800.0,Y
278695,2022-01-18,6589.0,1020,-16000.0,Y
384572,2022-01-19,6589.0,1020,10450.0,Y
...,...,...,...,...,...
208599,2022-12-02,6589.0,1020,-9998014.0,Y
515948,2022-12-29,6589.0,1020,-619747.0,Y
1041808,2022-12-16,6589.0,1020,1384625.0,Y
955996,2022-12-28,6589.0,1020,432375.0,Y
